In [1]:
import os
import numpy as np
from google import protobuf

import matplotlib.pyplot as plt
import urllib, json
import operator
%matplotlib inline

import re
import shutil
import os
from itertools import izip
import ast

from IPython.display import display, Markdown, Latex


import sys


from utilitites.parse_log import parse_log

import tabulate

# Fetch Completed jobs from DIGITS servers

In [10]:

data_folder = os.path.join(os.getcwd(), 'data')

#def fetch_digit_models(hosts = {0: 'http://localhost:5000', 1: 'http://localhost:5001'}.copy()):
#
#    try:
#        os.mkdir(data_folder)
#    except OSError as e:
#        pass
#
#    models = []
#    for i, addr in hosts.items():
#        response = urllib.urlopen(addr+ "/completed_jobs.json")
#        data = json.loads(response.read())
#        for m in data['models']:
#            m['node'] = i
#        models += data['models']
#
#    models = filter(lambda m: m['status'] != 'Error', models)
#    models = filter(lambda m: m['name'] not in ['LeNet', 'GoogLeNet'], models)
#    models_keys = sorted(list(set(reduce(operator.add, map(dict.keys, models)))))
#    return models, models_keys


In [16]:
print "\n".join(fetch_digit_models()[1])

accuracy (val) last
accuracy (val) max
accuracy (val) min
dataset_id
elapsed
epoch (train) last
epoch (train) max
epoch (train) min
epoch (val) last
epoch (val) max
epoch (val) min
framework
id
learning_rate (train) last
learning_rate (train) max
learning_rate (train) min
loss (train) last
loss (train) max
loss (train) min
loss (val) last
loss (val) max
loss (val) min
name
node
progress
sparkline
status
status_css
submitted
type


In [3]:
fetch_experiment_data = False
if fetch_experiment_data:
    # Download all network information
    for m in models:
        net_id = m['id']
        print "Fetching: ",net_id
        host = hosts[m['node']]
        
        net_url = host  + "/files/" + net_id + "/train_val.prototxt"
        net_str = urllib.urlopen(net_url).read()
        solver_url = host  + "/files/" + net_id + "/solver.prototxt"
        solver_str = urllib.urlopen(solver_url).read()
        log_url = host  + "/files/" + net_id + "/caffe_output.log"
        log_str = urllib.urlopen(log_url).read()
        

        net_path = os.path.join(data_folder, '%s_net.prototxt' % net_id)
        open(net_path, 'wb').write(net_str)
        solver_path = os.path.join(data_folder, '%s_solver.prototxt' % net_id)
        open(solver_path, 'wb').write(solver_str)
        log_path = os.path.join(data_folder, '%s_log.log' % net_id)
        open(log_path, 'wb').write(log_str)

In [19]:
get_net_info('20160703-164719-37a7')

{'batch_size': '32',
 'nparams': 61411783,
 'params': {'conv1': [(96, 15, 11, 11), (96,)],
  'conv2': [(256, 48, 5, 5), (256,)],
  'conv3': [(384, 256, 3, 3), (384,)],
  'conv4': [(384, 192, 3, 3), (384,)],
  'conv5': [(256, 384, 3, 3), (256,)],
  'fc6': [(4096, 9216), (4096,)],
  'fc7': [(4096, 4096), (4096,)],
  'fc8_species': [(967, 4096), (967,)]}}

In [83]:
print get_net_info('20160704-001701-30e0')['nparams']

119051207


In [193]:
def get_log_info(net_id):        
    log_path = os.path.join(data_folder, '%s_log.log' % net_id)
    log_str = open(log_path).read()
    
    result = []
    
    layer_number = -1
    name = None
    shapes = []
    active = False
    for l in log_str.split('\n'):
        if 'Creating training net' in l:
            active = True
        elif 'Creating test net' in l:
            active = False
        
        if not active:
            continue
        
        name_iter = re.search('net.cpp:153] Setting up (.*)', l)
        if name_iter:            
            new_name = name_iter.groups(0)[0]
            
            if name is not None:
                s = "UPDATE layers SET top_shape='%s' WHERE net_id='%s' AND layer_name like '%%%s'" %\
                    (str(shapes), net_id, name)
                result.append(s)         
            
            name = new_name
            layer_number += 1
            shapes = []
        shape_iter = re.search('Top shape: ([0-9 ]+)', l)
        if shape_iter:
            shapes.append(tuple(map(int,shape_iter.groups(0)[0].split()))[1:])
            
            
    return result
    return 0
    name_iter = re.finditer('net.cpp:153] Setting up (.*)', log_str)
    shape_iter = re.finditer('Top shape: ([0-9 ]+)', log_str)
    mem_iter = re.finditer('Memory required for data: (.*)', log_str)
    
    mem = dict()
    for n, m, s in izip(name_iter, mem_iter, shape_iter):
        print net_id, n.group(1), tuple(map(int,s.group(1).split()))
        mem[n.group(1)] = (int(m.group(1)) * 2**-20)
    return mem, parse_log(log_path)
#    return parse_log.parse_log(log_path)

print "\n".join(get_log_info('20160704-001701-30e0'));
pass
    
    

UPDATE layers SET top_shape='[(375, 47, 47)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%data'
UPDATE layers SET top_shape='[(1, 1, 1)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%label'
UPDATE layers SET top_shape='[(375, 8, 8)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%pool1'
UPDATE layers SET top_shape='[(4096,)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%fc6'
UPDATE layers SET top_shape='[(4096,)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%relu6'
UPDATE layers SET top_shape='[(4096,)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%drop6'
UPDATE layers SET top_shape='[(4096,)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%fc7'
UPDATE layers SET top_shape='[(4096,)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%relu7'
UPDATE layers SET top_shape='[(4096,)]' WHERE net_id='20160704-001701-30e0' AND layer_name like '%drop7'
UPDATE layers SET top_shape='[(967,)]' WHERE net

In [187]:
log_files = !ls data/*_log*


results = []
for f in log_files:
    results += get_log_info(os.path.basename(f)[:-8])    

results = [r + ";" for r in results]
res = "\n".join(results)
print res
open('top_shape.sql','w+').write(res)

UPDATE layers SET top_shape='[(3, 227, 227), ()]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%data';
UPDATE layers SET top_shape='[(96, 55, 55)]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%conv1';
UPDATE layers SET top_shape='[(96, 55, 55)]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%relu1';
UPDATE layers SET top_shape='[(96, 55, 55)]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%norm1';
UPDATE layers SET top_shape='[(96, 27, 27)]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%pool1';
UPDATE layers SET top_shape='[(256, 27, 27)]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%conv2';
UPDATE layers SET top_shape='[(256, 27, 27)]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%relu2';
UPDATE layers SET top_shape='[(256, 27, 27)]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%norm2';
UPDATE layers SET top_shape='[(256, 13, 13)]' WHERE net_id='20160624-121155-2c79' AND layer_name like '%p

    
def get_network_info(network):
    network = parse_prototxt(prototxt_str)
    res = dict()
    if not network:
        return res
        
    batch_size = (map(lambda l: l.get('batch_size',0),
                      filter(lambda l: l.get('phase')=='TRAIN', network)))
    res['batch_size'] = max(batch_size) if batch_size else None
    return res
    
    
get_network_info(get_network_prototxt(hosts[0], id))['batch_size']

In [88]:
for m in sorted(models, key=lambda m: m.get('accuracy (val) last'),):
    print "%50s" % m['name'], 
    print "%8.2f%%" % (100*m.get('accuracy (val) last',0)),
    print "%5i" % m['progress'],
    print "%s" % m['node'],
    info = get_net_info(m['id'])
    print "%4s" % (info['batch_size']),
    nparams = info['nparams']
    print "%8.1fe6" % (nparams * 10**-6),
    print "%8.1fMB" % (nparams * 4*2**-20),
    
    print 

                                        raw AlexFF     0.32%    52 1  200    653.9e6   2494.6MB
              a4_m2_s1s2s4s8_f2 i:conv5 p:2_3 adam     0.79%    56 0  200     38.3e6    146.1MB
                     a4_m3_s1s2s4_f2 i:conv3 p:2_3     4.12%    35 0   32    150.5e6    574.0MB
                               a1_m0_s1_f2 Alex1st     5.91%     8 0   64     61.3e6    233.7MB
                     a4_m3_s1s2s4_f2 i:conv3 p:2_3     6.30%   100 0   32    150.5e6    574.0MB
                           a4_m3_s1s2s4_f2 Alex4th     7.06%   100 0   32    107.7e6    411.0MB
                               a4_m1_s1_f2 Alex1st     7.66%    46 0   32     61.4e6    234.3MB
                       a4_m3_s1s2s4_f2 i:fc6 p:6_7     8.51%   100 0   64    119.1e6    454.1MB
                       a4_m2_s1s2_f2 i:conv3 p:5_6     9.59%   100 0   32    127.4e6    486.0MB
              a4_m2_s1s2_f2 i:conv3 p:5_6 batch<64    10.02%   100 0   32    127.4e6    486.0MB
                       a4_m2_s1s2_f2 i:c

In [153]:
def parse_name(name):
    components = name.split()
    d = dict()
    d['src'] = components[0]
    for c in components[1:]:
        if len(c.split(':')) == 2:
            k, v = c.split(':')
            d[k]=v
    return d
            
parse_name(models[0]['name'])    

{u'i': u'conv3', u'p': u'5_6', 'src': u'a4_m2_s1s2_f2'}

In [97]:


d = [{'i':42, 'k':5},{'i':10, 'v':12}]
s = tabulate.tabulate(d, tablefmt="pipe", headers={'i':'hev', 'k':None})
display(Markdown(s))


|   hev |   None |   v |
|------:|-------:|----:|
|    42 |      5 |     |
|    10 |        |  12 |

In [121]:

models_f = [m.copy() for m in models]
for m in models_f:
    del m['sparkline']
    del m['framework']
    del m['epoch (train) min']
    del m['epoch (train) max']
    del m['epoch (train) last']
    del m['epoch (val) min']
    del m['epoch (val) max']
    del m['epoch (val) last']
    del m['status_css']
    del m['status']
    del m['node']
    del m['type']
    del m['elapsed']
    del m['dataset_id']
    del m['submitted']
    del m['loss (val) max']
    del m['loss (train) max']
    del m['accuracy (val) min']
    


s = tabulate.tabulate(models_f, tablefmt="pipe", headers={})
#print (s)
display(Markdown(s))

|   accuracy (val) last |   learning_rate (train) max |   progress | id                   | name                                 |   loss (train) last |   loss (val) last |   loss (val) min |   accuracy (val) max |   learning_rate (train) last |   learning_rate (train) min |   loss (train) min |
|----------------------:|----------------------------:|-----------:|:---------------------|:-------------------------------------|--------------------:|------------------:|-----------------:|---------------------:|-----------------------------:|----------------------------:|-------------------:|
|            0.0959135  |                       0.01  |        100 | 20160708-110212-1143 | a4_m2_s1s2_f2 i:conv3 p:5_6          |         0.00196767  |           6.45885 |          5.29855 |           0.0979567  |                  2.06531e-05 |                 2.06531e-05 |        2.46287e-05 |
|            0.411298   |                       0.01  |        100 | 20160708-003123-0d2e | ok) raw Alex l:fc7 c:fc8             |         1.03617     |           3.04171 |          3.03845 |           0.414183   |                  5.94354e-05 |                 5.94354e-05 |        0.611328    |
|            0.10024    |                       0.01  |        100 | 20160706-065616-d8f8 | a4_m2_s1s2_f2 i:conv3 p:5_6 batch<64 |         0.02067     |           6.3088  |          5.24854 |           0.101322   |                  2.06531e-05 |                 2.06531e-05 |        2.87446e-05 |
|            0.139784   |                       0.01  |        100 | 20160705-221918-6a5b | a4_m3_s1s2s4_f2 i:conv5 p:5_6        |         0.0979416   |           6.5157  |          5.01169 |           0.140144   |                  2.06475e-05 |                 2.06475e-05 |        0.000411439 |
|            0.202524   |                       0.01  |        100 | 20160705-221756-e7b1 | a4_m2_s1s2_f2 i:conv1                |         0.00312502  |           8.48966 |          4.38468 |           0.203365   |                  2.06475e-05 |                 2.06475e-05 |        0.00042857  |
|            0.165264   |                       0.01  |        100 | 20160705-221737-ca2c | a4_m2_s1s2_f2 i:conv2 p:3_4          |         0.00125641  |           6.14472 |          4.79586 |           0.165986   |                  2.06475e-05 |                 2.06475e-05 |        5.82855e-05 |
|            0.186178   |                       0.001 |        100 | 20160705-220738-8a08 | a4_m2_s1s2_f2 i:conv5 p:5_6          |         0.00528707  |           5.59722 |          4.59859 |           0.18726    |                  2.06531e-06 |                 2.06531e-06 |        0.00381639  |
|            0.0629808  |                       0.01  |        100 | 20160705-215037-3a1c | a4_m3_s1s2s4_f2 i:conv3 p:2_3        |         0.000134584 |           6.75043 |          5.47281 |           0.0647837  |                  0.000168912 |                 0.000168912 |        4.08556e-05 |
|            0.214762   |                       0.001 |        100 | 20160705-214944-ddc0 | a4_m2_s1s2s4s8_f2 i:conv5 p:1_2      |         0.857368    |           4.70575 |          4.39423 |           0.218095   |                  2.06569e-06 |                 2.06569e-06 |        0.67938     |
|            0.00785714 |                       0.01  |         56 | 20160705-130445-443c | a4_m2_s1s2s4s8_f2 i:conv5 p:2_3 adam |         6.68809     |           6.58626 |          6.58569 |           0.00809524 |                  0.01        |                 0.01        |        6.50862     |
|            0.120952   |                       0.01  |         54 | 20160705-071507-f290 | a4_m2_s1s2s4s8_f2 i:conv4            |         0.159153    |           6.7545  |          5.01313 |           0.120952   |                  0.00109856  |                 0.00109856  |        0.0261466   |
|            0.180714   |                       0.01  |        100 | 20160704-232348-6577 | a4_m2_s1s2s4s8_f2 i:conv3            |         0.00398609  |           5.65547 |          4.53846 |           0.182143   |                  0.00016945  |                 0.00016945  |        4.54235e-05 |
|            0.242976   |                       0.005 |        100 | 20160704-171221-2222 | a4_m2_s1s2s4s8_f2 i:conv5 p:2_3      |         0.143595    |           5.41082 |          4.27598 |           0.244048   |                  1.0304e-05  |                 1.0304e-05  |        0.0938605   |
|            0.0705529  |                       0.01  |        100 | 20160704-171110-ef79 | a4_m3_s1s2s4_f2 Alex4th              |         0.827191    |           9.26239 |          5.67566 |           0.0716346  |                  2.06723e-05 |                 2.06723e-05 |        0.00193246  |
|            0.211905   |                       0.01  |        100 | 20160704-162710-6dc9 | a4_m2_s1s2s4s8_f2 i:conv5            |         1.92575     |           4.1829  |          4.02057 |           0.213214   |                  2.06081e-05 |                 2.06081e-05 |        1.63758     |
|            0.219762   |                       0.001 |         80 | 20160704-154710-e898 | a4_m2_s1s2s4s8_f2 i:conv5            |         0.481317    |           4.91698 |          4.4165  |           0.222619   |                  7.11014e-06 |                 7.11014e-06 |        0.325209    |
|            0.209524   |                       0.001 |         29 | 20160704-152854-7d95 | a4_m2_s1s2s4s8_f2 i:conv5 p:1_2      |         0.971091    |           4.68772 |          4.3705  |           0.209881   |                  0.000161972 |                 0.000161972 |        0.88843     |
|            0.421875   |                       0.01  |        100 | 20160704-065721-1c42 | raw Alex l:conv5 c:conv6             |         0.672011    |           2.79306 |          2.77809 |           0.423077   |                  5.94354e-05 |                 5.94354e-05 |        0.364037    |
|            0.041226   |                       0.01  |         35 | 20160704-001911-eb45 | a4_m3_s1s2s4_f2 i:conv3 p:2_3        |         0.195313    |           6.74296 |          5.47261 |           0.0526442  |                  0.00240597  |                 0.00240597  |        0.195313    |
|            0.140238   |                       0.01  |         54 | 20160704-001743-6578 | a4_m3_s1s2s4_f2 i:conv3 p:5_6        |         0.0127431   |           6.35072 |          4.7289  |           0.140238   |                  0.00110414  |                 0.00110414  |        0.00165712  |
|            0.0850962  |                       0.001 |        100 | 20160704-001701-30e0 | a4_m3_s1s2s4_f2 i:fc6 p:6_7          |         0.201197    |           7.01671 |          5.78282 |           0.0858173  |                  1.69382e-05 |                 1.69382e-05 |        0.0827919   |
|            0.176548   |                       0.001 |        100 | 20160704-001627-a4de | a4_m3_s1s2s4_f2 i:conv5 p:4_5        |         0.175044    |           5.34903 |          4.55009 |           0.180833   |                  2.06545e-06 |                 2.06545e-06 |        0.0917473   |
|            0.213582   |                       0.01  |         14 | 20160704-001413-898d | caffenet_trans_cleanFF_lockConv      |         4.57945     |           4.0749  |          4.0749  |           0.213582   |                  0.00491532  |                 0.00491532  |        4.19793     |
|            0.13726    |                       0.01  |        100 | 20160703-232402-8605 | a4_m3_s1s2s4_f2 i:conv5 p:5_6        |         3.77535     |           4.52744 |          4.52744 |           0.13726    |                  2.08356e-05 |                 2.08356e-05 |        3.10701     |
|            0.102043   |                       0.001 |         65 | 20160703-211919-c6bb | a4_m2_s1s2_f2 i:conv5 p:5_6          |         4.98111     |           4.9674  |          4.9674  |           0.102043   |                  1.80811e-05 |                 1.80811e-05 |        3.67212     |
|            0.158173   |                       0.01  |        100 | 20160703-195348-6004 | a4_m2_s1s2_f2 i:conv3 p:5_6          |         2.63168     |           4.43479 |          4.42925 |           0.158173   |                  2.08356e-05 |                 2.08356e-05 |        2.1885      |
|            0.149519   |                       0.01  |        100 | 20160703-195208-cdcf | a4_m2_s1s2_f2 i:conv2 p:3_4          |         2.87312     |           4.42293 |          4.40505 |           0.149639   |                  2.08356e-05 |                 2.08356e-05 |        2.4489      |
|            0.103365   |                       0.01  |        100 | 20160703-184038-b35f | a4_m2_s1s2_f2 i:conv1                |         4.07156     |           4.59655 |          4.59655 |           0.103365   |                  2.08356e-05 |                 2.08356e-05 |        3.7671      |
|            0.0765625  |                       0.01  |         46 | 20160703-165031-26a6 | a4_m1_s1_f2 Alex1st                  |         4.50514     |           4.89094 |          4.89094 |           0.0765625  |                  0.000575371 |                 0.000575371 |        4.38977     |
|            0.128365   |                       0.01  |        100 | 20160703-164719-37a7 | a4_m1_s1_f2 Alex1st                  |         4.18931     |           4.53678 |          4.53678 |           0.128365   |                  2.08356e-05 |                 2.08356e-05 |        3.67938     |
|            0.0591346  |                       0.01  |          8 | 20160703-150520-3997 | a1_m0_s1_f2 Alex1st                  |         4.88616     |           5.1691  |          5.1691  |           0.0591346  |                  0.00624521  |                 0.00624521  |        4.60131     |
|            0.288452   |                       0.01  |        100 | 20160625-083048-3086 | raw Alex c:conv1 v2                  |         0.113556    |           5.60039 |          4.03166 |           0.288929   |                  2.06545e-05 |                 2.06545e-05 |        0.0194873   |
|            0.473929   |                       0.01  |        100 | 20160625-001322-6968 | raw Alex t:conv4 c:conv5             |         0.0422956   |           3.79226 |          3.06751 |           0.476548   |                  2.06545e-05 |                 2.06545e-05 |        0.0041855   |
|            0.505833   |                       0.01  |        100 | 20160624-182808-811a | raw Alex t:fc7 c:fc8                 |         0.00938635  |           3.25371 |          3.01448 |           0.507381   |                  5.94831e-05 |                 5.94831e-05 |        0.00152035  |
|            0.2625     |                       0.01  |        100 | 20160624-121155-2c79 | raw Alex c:conv1                     |         1.53223     |           4.01136 |          3.85539 |           0.2625     |                  1e-05       |                 1e-05       |        1.06588     |
|            0.438941   |                       0.01  |        100 | 20160708-003847-e5ae | raw l:fc7 c:fc8                      |         1.36492     |           2.59994 |          2.59225 |           0.440118   |                  5.94101e-05 |                 5.94101e-05 |        1.14179     |
|            0.356429   |                       0.01  |        100 | 20160705-214821-0e83 | raw l:conv1 c:conv2                  |         0.178203    |           4.64287 |          3.6101  |           0.35881    |                  5.94614e-05 |                 5.94614e-05 |        0.0301545   |
|            0.459881   |                       0.01  |        100 | 20160705-214810-44a7 | raw l:conv2 c:conv3                  |         0.0990544   |           3.81492 |          3.00011 |           0.459881   |                  5.94614e-05 |                 5.94614e-05 |        0.0333782   |
|            0.483929   |                       0.01  |        100 | 20160705-214758-e0a4 | raw l:conv3 c:conv4                  |         0.0749985   |           3.46183 |          2.84925 |           0.485119   |                  5.94614e-05 |                 5.94614e-05 |        0.0193339   |
|            0.458809   |                       0.01  |        100 | 20160705-214733-e267 | raw t:conv2 c:conv3                  |         0.147325    |           3.95081 |          3.06082 |           0.458809   |                  5.94614e-05 |                 5.94614e-05 |        0.0406099   |
|            0.341429   |                       0.01  |        100 | 20160705-214721-4d47 | raw t:conv1 c:conv2                  |         0.153589    |           4.81878 |          3.76925 |           0.343452   |                  5.94614e-05 |                 5.94614e-05 |        0.0539525   |
|            0.485714   |                       0.01  |        100 | 20160705-214703-ac3e | raw t:conv3 c:conv4                  |         0.10233     |           3.67421 |          2.86813 |           0.485714   |                  5.94614e-05 |                 5.94614e-05 |        0.0172614   |
|            0.480714   |                       0.01  |        100 | 20160705-131412-ff07 | raw l:conv4 c:conv5                  |         0.0488859   |           3.40761 |          2.88972 |           0.480714   |                  5.93634e-05 |                 5.93634e-05 |        0.0190695   |
|            0.526548   |                       0.01  |        100 | 20160705-091349-2744 | raw t:fc7 l:conv4 c:fc8              |         0.0366117   |           3.06933 |          2.78659 |           0.526667   |                  5.93634e-05 |                 5.93634e-05 |        0.00259915  |
|            0.503452   |                       0.01  |         17 | 20160705-075154-d89f | raw l:conv4 c:conv5                  |         0.177353    |           2.61274 |          2.52025 |           0.503452   |                  0.00420245  |                 0.00420245  |        0.172153    |
|            0.439167   |                       0.01  |         32 | 20160705-072003-d85d | raw l:fc7 c:fc8                      |         1.37499     |           2.60744 |          2.60744 |           0.439167   |                  0.0019149   |                 0.0019149   |        1.14102     |
|            0.340714   |                       0.01  |        100 | 20160704-232921-df78 | caffenet_trans_cleanFF_lockConv_new  |         3.30928     |           3.48987 |          3.48413 |           0.342381   |                  5.93808e-05 |                 5.93808e-05 |        2.78537     |
|            0.00321429 |                       0.001 |         52 | 20160704-144437-f9a3 | raw AlexFF                           |         6.85933     |           6.85683 |          6.85683 |           0.00345238 |                  0.0001      |                 0.0001      |        6.84831     |
|            0.499524   |                       0.01  |        100 | 20160704-104545-8c3e | caffenet_trans_cleanFF_lockConv_new  |         0.141801    |           2.72975 |          2.61158 |           0.500595   |                  5.93808e-05 |                 5.93808e-05 |        0.0797321   |
|            0.472957   |                       0.01  |        100 | 20160704-090616-ca2e | AlexNetTransfer                      |         0.282563    |           2.87541 |          2.75557 |           0.475601   |                  1e-05       |                 1e-05       |        0.188101    |

In [153]:
key_map = {'id':'net_id'}

for m in models_f:
    m = m.copy()
    columns = []
    values = []
    for k, v in m.items():
        k = key_map[k] if k in key_map else k
        k = "_".join(k.split())
        k = re.sub('[\(\)]','', k)
        
        columns.append("'%s'"%str(k))
        if type(v) is float:
            values.append("%f" % v)
        elif type(v) is int:
            values.append("%i" % v)                
        else:
            values.append("'%s'" % str(v))
            
    q = "INSERT INTO networks (%s)\nVALUES (%s);" % (",".join(columns), ",".join(values))
    print q

INSERT INTO networks ('name','accuracy_val_last','accuracy_val_max','learning_rate_train_min','loss_val_last','learning_rate_train_max','loss_val_min','loss_train_last','progress','learning_rate_train_last','net_id','loss_train_min')
VALUES ('a4_m2_s1s2_f2 i:conv3 p:5_6',0.095913,0.097957,0.000021,6.458850,0.010000,5.298550,0.001968,100,0.000021,'20160708-110212-1143',0.000025);
INSERT INTO networks ('name','accuracy_val_last','accuracy_val_max','learning_rate_train_min','loss_val_last','learning_rate_train_max','loss_val_min','loss_train_last','progress','learning_rate_train_last','net_id','loss_train_min')
VALUES ('ok) raw Alex l:fc7 c:fc8',0.411298,0.414183,0.000059,3.041710,0.010000,3.038450,1.036170,100,0.000059,'20160708-003123-0d2e',0.611328);
INSERT INTO networks ('name','accuracy_val_last','accuracy_val_max','learning_rate_train_min','loss_val_last','learning_rate_train_max','loss_val_min','loss_train_last','progress','learning_rate_train_last','net_id','loss_train_min')
VALUE

In [144]:
212064000. / (64*375*47*47)

4.0

In [54]:
821694400 / 921600.

891.5954861111111

In [112]:
# Legacy

total = 0
for k, ps in sorted(param.items()):
    size = reduce(operator.add, [reduce(operator.mul,p) for p in ps])
    total += size      
    print "%12s, %5.1fM, %6.4f GB" % (k, size*10**-6, size * 4 * 2**-30)
    
print "-" * 32
print "%12s, %5.1fM, %6.4f GB" % ("total",total*10**-6, total * 4 * 2**-30)
print "=" * 32

NameError: name 'param' is not defined